对于文本来说，大语言模型的本质是在提示词后面继续生成新的文本，因此提示词是大语言模型的核心，它决定了语言模型的行为和输出。虽然提示词完全可以手搓，但为了方便，LangChain 提供了一些提示词模板工具，用户只需先制作模板，后续修改一些参数即可得到对应的提示词。LangChain 提供了 3 种常用的提示词模板，分别是：
1. `PromptTemplate`：最基本的提示词模板，用户可以自定义提示词的模板，并通过传入参数得到对应的提示词。
2. `ChatPromptTemplate`：用于聊天模型的提示词模板，可以生成聊天模型的例子，并通过传入参数得到对应的提示词。
3. `FewShotPromptTemplate`：用于 few-shot learning 的提示词模板，可以生成 few-shot learning 的例子，并通过传入参数得到对应的提示词。
# `PromptTemplate`
提示词模板接受一个字典（`invoke` 方法）或命名参数（`format` 方法），键是模版参数，值是对应的值。

In [15]:
from langchain_core.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="qwen2.5:0.5b")

template = '''
Tell me a joke about {topic}
'''

prompt_template = PromptTemplate.from_template(template)
print(prompt_template.invoke({'topic':"cats"})) # 这两种都可以
print(prompt_template.format(topic="cats")) # 这两种都可以

prompt = prompt_template.format(topic="cats")
print(llm.invoke(prompt))

text='\nTell me a joke about cats\n'

Tell me a joke about cats

Sure! Here's a joke about cats:
Why did the cat break up with the cat?
Because it found out she was really good at throwing cans!
I hope that makes for a funny response! How else could I help you today?


也可以显式写出变量的名称。

In [16]:
prompt = PromptTemplate(
    input_variables=["topic"], 
    template="Tell me a joke about {topic}"
)

print(prompt.invoke({"topic":"cats"}))
print(prompt.format(topic="cats"))

prompt = prompt.format(topic="cats")
print(llm.invoke(prompt))

text='Tell me a joke about cats'
Tell me a joke about cats
Sure! Here's a cat joke for you:

Why did the cat break up with the tree?

Because it wanted to have a catnap on the ground!

This is a playful and lighthearted way of making a point, as if you're trying to be a bit naughty. I hope this makes your day a little less stressful!


`ChatPromptTemplate`
对话通常由不同角色的对话信息组成，例如系统信息、用户信息和助手信息，但不能一概而论，系统信息和用户信息也经常合并或缺失。构建对话提示词模板既直接接受一个结构化数据：

In [17]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])

print(prompt_template.invoke({"topic": "cats"}))

prompt = prompt.format(topic="cats")
print(llm.invoke(prompt))

messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about cats', additional_kwargs={}, response_metadata={})]
Why did the cat break up with the moon? Because it was too high up!


也可以接受提示词模板封装：

In [18]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_template = "You are a helpful assistant"
system_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = "Tell me a joke about {topic}?"
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

prompt_template = ChatPromptTemplate.from_messages([system_prompt, human_prompt])
print(prompt_template.format_prompt(topic="cats"))

prompt = prompt.format(topic="cats")
print(llm.invoke(prompt))

messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about cats?', additional_kwargs={}, response_metadata={})]
Here's a cat-related joke for you:

Why did the cat break up with the apple?

Because it wanted to have a party!


# `FewShotPromptTemplate`
提供若干个用例可以提高模型的性能，指导大语言模型按照用户的期望生成文本。LangChain 提供了若干个用例选择器来根据用户的需求选择合适的用例。
- `SemanticSimilarityExampleSelector`：调用大语言模型，根据语义相似度找出最相关的用例
- `LengthBasedExampleSelector`：根据长度选择用例，长用例被选择的概率更小，短用例被选择的概率更大
- `MaxMarginalRelevanceExampleSelector`：根据最大边际相关性找出最相关的用例
- `NGramOverlapExampleSelector`：使用 ngram 重合度来选择最相关的例子

这里以 `SemanticSimilarityExampleSelector` 为例，展示如何构建少样本学习的提示词。

In [29]:
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.prompts.example_selector import (SemanticSimilarityExampleSelector, 
                                                LengthBasedExampleSelector, 
                                                MaxMarginalRelevanceExampleSelector, 
                                                NGramOverlapExampleSelector)

samples = [
  {
    "flower_type": "雏菊",
    "occasion": "友情",
    "ad_copy": "雏菊象征着纯真和真挚，是你向朋友表达友谊和温暖的贴心选择。"
  },
  {
    "flower_type": "勿忘我",
    "occasion": "离别",
    "ad_copy": "勿忘我代表着珍重与回忆，是你在离别之际表达深情的理想花卉。"
  },
  {
    "flower_type": "紫罗兰",
    "occasion": "怀念",
    "ad_copy": "紫罗兰象征着怀念与永恒，是你缅怀过去美好时光的优雅选择。"
  },
  {
    "flower_type": "郁金香",
    "occasion": "祝福",
    "ad_copy": "郁金香象征着美好与幸福，是你送上诚挚祝福的理想选择。"
  },
  {
    "flower_type": "勿忘我",
    "occasion": "回忆",
    "ad_copy": "勿忘我寓意着真挚的情感，是你寄托思念和回忆的最佳选择。"
  }
]

首先创建一个提示词模板。

In [30]:
prompt_sample = PromptTemplate(input_variables=["flower_type", "occasion", "ad_copy"], 
                               template="鲜花类型: {flower_type}\n场合: {occasion}\n文案: {ad_copy}")
print(prompt_sample.format(**samples[0]))

鲜花类型: 雏菊
场合: 友情
文案: 雏菊象征着纯真和真挚，是你向朋友表达友谊和温暖的贴心选择。


`FewShotPromptTemplate` 包含若干个参数，主要参数有：
- `example_prompt`：用来格式化每条用例的模板
- `example_selector`：用例选择器
- `examples`：具体用例
- `prefix`：前缀，用来提示模型如何使用数据
- `suffix`：后缀，用来提示模型输出的格式
- `input_variables`：输入变量，用来指定模板中需要填充的变量
- `partial_variables`：部分变量，用来指定模板中需要填充的变量，但值不是在模板中指定的，而是通过其他方式提供的

In [31]:
from langchain.prompts import PromptTemplate

prompt_sample = PromptTemplate(template='鲜花类型：{flower_type}\n场合：{occasion}\n文案：{ad_copy}', 
                               input_variables=['flower_type', 'occasion', 'ad_copy'])
print(prompt_sample)

input_variables=['ad_copy', 'flower_type', 'occasion'] input_types={} partial_variables={} template='鲜花类型：{flower_type}\n场合：{occasion}\n文案：{ad_copy}'


In [32]:
from langchain.prompts.few_shot import FewShotPromptTemplate
prompt = FewShotPromptTemplate(
    examples=samples,
    example_prompt=prompt_sample,
    prefix='以下是鲜花类型和场合的对应关系，请根据occasion给出对应的鲜花类型：',
    suffix='鲜花类型：{flower_type}\n场合：{occasion}',
    input_variables=['flower_type', 'occasion']
)
print(prompt.format(flower_type='玫瑰', occasion='生日'))

以下是鲜花类型和场合的对应关系，请根据occasion给出对应的鲜花类型：

鲜花类型：雏菊
场合：友情
文案：雏菊象征着纯真和真挚，是你向朋友表达友谊和温暖的贴心选择。

鲜花类型：勿忘我
场合：离别
文案：勿忘我代表着珍重与回忆，是你在离别之际表达深情的理想花卉。

鲜花类型：紫罗兰
场合：怀念
文案：紫罗兰象征着怀念与永恒，是你缅怀过去美好时光的优雅选择。

鲜花类型：郁金香
场合：祝福
文案：郁金香象征着美好与幸福，是你送上诚挚祝福的理想选择。

鲜花类型：勿忘我
场合：回忆
文案：勿忘我寓意着真挚的情感，是你寄托思念和回忆的最佳选择。

鲜花类型：玫瑰
场合：生日


`FewShotPromptTemplate` 需要借助词嵌入模型、向量数据库和大语言模型完成对语义相近的 few-shot 示例的检索，从而完成 few-shot 提示词模板的构建。

In [33]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama.llms import OllamaLLM
from langchain.vectorstores import FAISS

embedding = HuggingFaceEmbeddings(model_name='BAAI/bge-small-zh-v1.5')
llm = OllamaLLM(model="qwen2.5:0.5b")

example_selector = SemanticSimilarityExampleSelector.from_examples(
    samples, embedding, FAISS, k=1
)

prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=prompt_sample,
    suffix='鲜花类型：{flower_type}\n场合：{occasion}',
    input_variables=['flower_type', 'occasion']
)
print(prompt.format(flower_type='红玫瑰', occasion='爱情'))

鲜花类型：郁金香
场合：祝福
文案：郁金香象征着美好与幸福，是你送上诚挚祝福的理想选择。

鲜花类型：红玫瑰
场合：爱情


这样就生成了一条包含于查询相关的用例的提示词。